In [1]:
import torchvision
from torchvision import transforms,datasets
import torch
from torch.utils.data import DataLoader
from torch.autograd import Variable

import matplotlib.pyplot as plt

In [ ]:
batch_size = 64
learning_rate = 1e-3
num_epoches = 100

In [3]:
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor())

In [4]:
transform=transforms.Compose([transforms.ToTensor(),
                             transforms.Normalize(mean=[0.5,0.5,0.,5],std=[0.5,0.5,0.5])])

In [5]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [ ]:
images, labels =next(iter(train_loader))
img = torchvision.utils.make_grid(images)
img = img.numpy().transpose(1,2,0)
std = [0.5,0.5,0.5]
mean = [0.5,0.5,0.5]
img = img*std + mean
print([labels[i] for i in range(64)])
plt.imshow(img)

In [8]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [9]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 输入1通道，输出10通道，kernel 5*5
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        # fully connect
        self.fc = nn.Linear(320, 10)

    def forward(self, x):
        in_size = x.size(0) 
        x = F.relu(self.mp(self.conv1(x)))
        x = F.relu(self.mp(self.conv2(x)))
        x = x.view(in_size, -1) 
        x = self.fc(x)
        return F.log_softmax(x)


model = Net()

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [10]:
def train(epoch):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def loss():
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


for epoch in range(1, 10):
    train(epoch)
    loss()

F:\Anaconda\envs\tensorFlow\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.311006
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.627008
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.221774
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.319970
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.233606


F:\Anaconda\envs\tensorFlow\lib\site-packages\ipykernel_launcher.py:19: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
F:\Anaconda\envs\tensorFlow\lib\site-packages\torch\nn\functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.1788, Accuracy: 9466/10000 (94%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.300426
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.173550
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.164781
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.195736
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.059441

Test set: Average loss: 0.1019, Accuracy: 9701/10000 (97%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.194260
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.103434
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.072368
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.143629
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.069597

Test set: Average loss: 0.0838, Accuracy: 9743/10000 (97%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.055716
Train Epoch: 4 [12800/60000 (21%)]	Loss: 0.099553
Train Epoch: 4 [25600/60000 (43%)]	Loss: 0.078719
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.026055
Train Epoch: 4 [51200/60000 (85%)]	Loss: 0.046294

Test set: Average loss: 0.0808, Accuracy: 9754/10000 (97%)

Train Epoch: 5 [0/600

KeyboardInterrupt: 